In [20]:
from functools import wraps
import time
from typing import Optional, Tuple

import numpy as np
import polars as pl
from scipy.sparse import coo_matrix, csr_matrix


In [22]:
interactions = pl.read_csv("../data/ml-1m/interactions_1k.csv", schema={"user_id": pl.Int32, "item_id": pl.Int32})
interactions


user_id,item_id
i32,i32
3391,2987
3391,1248
3391,1249
3391,719
3391,574
3391,2050
3391,2051
3391,3791
3391,2052


In [23]:
class OrdinalEncoder:
    def __init__(self) -> None:
        super().__init__()

    def fit(self, df: pl.DataFrame, column: str) -> "OrdinalEncoder":
        self._mapper = (
            df[[column]].unique()
            .sort(column)
            .with_row_count("__index__")
            .with_columns(pl.col("__index__").cast(pl.Int32))
        )
        self._fit_column = column
        return self

    def transform(self, df: pl.DataFrame, column: str) -> pl.DataFrame:
        df = (
            df
            .rename({column: self._fit_column})
            .join(self._mapper, on=self._fit_column, how="left")
            .drop(self._fit_column)
            .rename({"__index__": column})
        )
        return df

    def inverse_transform(self, df: pl.DataFrame, column: str) -> pl.DataFrame:
        df = (
            df
            .rename({column: "__index__"})
            .join(
                self._mapper.rename({self._fit_column: column}),
                on="__index__",
                how="left",
            )
            .drop(f"__index__")
        )
        return df

    def fit_transform(self, df: pl.DataFrame, source: str, target: Optional[str] = None) -> pl.DataFrame:
        return self.fit(df, source).transform(df, target or source)


user_id_encoder = OrdinalEncoder()
item_id_encoder = OrdinalEncoder()

interactions_encoded = user_id_encoder.fit_transform(interactions, "user_id")
interactions_encoded = item_id_encoder.fit_transform(interactions_encoded, "item_id")


In [24]:
interactions_encoded


user_id,item_id
i32,i32
563,2591
563,1054
563,1055
563,638
563,539
563,1726
563,1727
563,3298
563,1728


In [25]:
user_idx = interactions_encoded["user_id"].to_numpy()
item_idx = interactions_encoded["item_id"].to_numpy()

n_users = user_idx.max() + 1
n_items = item_idx.max() + 1

user_item_coo = coo_matrix(
    (
        np.ones_like(user_idx, dtype=np.float32),
        (user_idx, item_idx),
    ),
    shape=(n_users, n_items),
    dtype=np.float32,
)

# Make sure we have canonical format
user_item_coo.sum_duplicates()
user_item_coo


<1000x3446 sparse matrix of type '<class 'numpy.float32'>'
	with 172569 stored elements in COOrdinate format>

In [28]:
user_item_csr = user_item_coo.tocsr()
user_item_csr


<1000x3446 sparse matrix of type '<class 'numpy.float32'>'
	with 172569 stored elements in Compressed Sparse Row format>

In [29]:
# Normalize each row by non zero element count in the row
nnz_per_row = user_item_csr.indptr[1:] - user_item_csr.indptr[:-1]
user_item_csr.data /= np.repeat(nnz_per_row, nnz_per_row)

# Make sure data is normalize
user_item_csr.sum(axis=1)[:10]


matrix([[1.0000002],
        [1.       ],
        [1.       ],
        [0.9999999],
        [1.       ],
        [1.0000001],
        [0.9999998],
        [0.9999999],
        [0.9999999],
        [1.       ]], dtype=float32)

In [30]:
distances = user_item_csr @ user_item_csr.T
distances


<1000x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 958194 stored elements in Compressed Sparse Row format>

In [31]:
distances_dense = distances.toarray()
top_user_indices = np.argsort(-distances_dense, axis=1)[:, 1 : 1 + 10].astype(np.int32)
top_similarity = np.take(distances_dense, top_user_indices)

top_similar_users = pl.DataFrame({
    "user_id": np.arange(0, n_users, dtype=np.int32),
    "similar_user_id": top_user_indices,
    "similarity": top_similarity,
})
top_similar_users = top_similar_users.explode("similar_user_id", "similarity")
top_similar_users


user_id,similar_user_id,similarity
i32,i32,f32
0,300,0.006173
0,629,0.006066
0,717,0.005258
0,255,0.004986
0,567,0.004986
0,975,0.00481
0,599,0.004522
0,373,0.004517
0,761,0.00434


In [14]:
top_similar_users = user_id_encoder.inverse_transform(top_similar_users, "user_id")
top_similar_users = user_id_encoder.inverse_transform(top_similar_users, "similar_user_id")
top_similar_users


similarity,user_id,similar_user_id
f32,i32,i32
0.006173,13,1849
0.006066,13,3730
0.005258,13,4320
0.004986,13,1558
0.004986,13,3404
0.00481,13,5931
0.004522,13,3555
0.004517,13,2291
0.00434,13,4668


In [32]:
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper


def interaction_to_csr(interactions: pl.DataFrame) -> csr_matrix:
    user_idx = interactions["user_id"].to_numpy()
    item_idx = interactions["item_id"].to_numpy()

    n_users = user_idx.max() + 1
    n_items = item_idx.max() + 1

    user_item_coo = coo_matrix(
        (
            np.ones_like(user_idx, dtype=np.bool_),
            (user_idx, item_idx),
        ),
        shape=(n_users, n_items),
        dtype=np.bool_,
    )
    user_item_coo.sum_duplicates()

    user_item_csr = user_item_coo.astype(np.float32).tocsr()
    nnz_per_row = user_item_csr.indptr[1:] - user_item_csr.indptr[:-1]
    user_item_csr.data /= np.repeat(nnz_per_row, nnz_per_row).astype(np.float32)

    return user_item_csr


def top_distances(distances: csr_matrix, k: int) -> Tuple[np.ndarray, np.ndarray]:
    distances_dense = distances.toarray()
    top_user_indices = np.argsort(-distances_dense, axis=1)[:, 1 : 1 + k].astype(np.int32)
    top_similarity = np.take(distances_dense, top_user_indices)
    return top_user_indices, top_similarity


@timeit
def user_similarity(interactions: pl.DataFrame) -> pl.DataFrame:
    # Encode IDs to indices
    user_id_encoder = OrdinalEncoder()
    item_id_encoder = OrdinalEncoder()
    interactions_encoded = user_id_encoder.fit_transform(interactions, "user_id")
    interactions_encoded = item_id_encoder.fit_transform(interactions_encoded, "item_id")

    # Convert interactions to CSR
    interactions_csr = interaction_to_csr(interactions_encoded)

    # Compute full distances matrix using matrix multiplication
    distances = interactions_csr @ interactions_csr.T

    # Get Top-K indices and distances from full-distance matrix
    top_user_indices, top_similarity = top_distances(distances, k=10)

    # Wrap indices and distances to data frame
    top_similar_users = pl.DataFrame({
        "user_id": np.arange(0, top_user_indices.shape[0], dtype=np.int32),
        "similar_user_id": top_user_indices,
        "similarity": top_similarity,
    })
    top_similar_users = top_similar_users.explode("similar_user_id", "similarity")

    # Decode indices to IDs
    top_similar_users = user_id_encoder.inverse_transform(top_similar_users, "user_id")
    top_similar_users = user_id_encoder.inverse_transform(top_similar_users, "similar_user_id")

    return top_similar_users


In [33]:
user_similarity(interactions)


Function user_similarity Took 0.3064 seconds


similarity,user_id,similar_user_id
f32,i32,i32
0.006173,13,1849
0.006066,13,3730
0.005258,13,4320
0.004986,13,1558
0.004986,13,3404
0.00481,13,5931
0.004522,13,3555
0.004517,13,2291
0.00434,13,4668


In [34]:
interactions_1k = pl.read_csv("../data/ml-1m/interactions_1k.csv", schema={"user_id": pl.Int32, "item_id": pl.Int32})
interactions_2k = pl.read_csv("../data/ml-1m/interactions_2k.csv", schema={"user_id": pl.Int32, "item_id": pl.Int32})
interactions_5k = pl.read_csv("../data/ml-1m/interactions_5k.csv", schema={"user_id": pl.Int32, "item_id": pl.Int32})


In [35]:
user_similarity(interactions_1k)


Function user_similarity Took 0.2517 seconds


similarity,user_id,similar_user_id
f32,i32,i32
0.006173,13,1849
0.006066,13,3730
0.005258,13,4320
0.004986,13,1558
0.004986,13,3404
0.00481,13,5931
0.004522,13,3555
0.004517,13,2291
0.00434,13,4668


In [38]:
user_similarity(interactions_2k)


Function user_similarity Took 0.7450 seconds


similarity,user_id,similar_user_id
f32,i32,i32
0.018634,4,2347
0.014778,4,3535
0.014157,4,3461
0.013889,4,5388
0.013889,4,1771
0.013605,4,3616
0.013289,4,1349
0.012897,4,4966
0.012821,4,1558


In [39]:
user_similarity(interactions_5k)


Function user_similarity Took 3.8924 seconds


similarity,user_id,similar_user_id
f32,i32,i32
0.00891,1,5343
0.007898,1,5190
0.007383,1,1283
0.006951,1,681
0.006604,1,5525
0.006563,1,5320
0.006563,1,2799
0.006289,1,317
0.006003,1,417
